In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

import utils.data_processing_bronze_feature
import utils.data_processing_silver_feature
import utils.data_processing_gold_feature





In [2]:
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")  

snapshot_date_str = "2023-01-01"

start_date_str = "2023-01-01"
end_date_str = "2024-12-01"

def generate_first_of_month_dates(start_date_str, end_date_str):
    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # List to store the first of month dates
    first_of_month_dates = []

    # Start from the first of the month of the start_date
    current_date = datetime(start_date.year, start_date.month, 1)

    while current_date <= end_date:
        # Append the date in yyyy-mm-dd format
        first_of_month_dates.append(current_date.strftime("%Y-%m-%d"))
        
        # Move to the first of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return first_of_month_dates

dates_str_lst = generate_first_of_month_dates(start_date_str, end_date_str)
print(dates_str_lst)



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/20 15:00:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


['2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01', '2024-05-01', '2024-06-01', '2024-07-01', '2024-08-01', '2024-09-01', '2024-10-01', '2024-11-01', '2024-12-01']


In [3]:
bronze_lms_directory_loan = "datamart/bronze/lms_loan/"
bronze_lms_directory_clickstream = "datamart/bronze/lms_clickstream/"
bronze_lms_directory_attributes = "datamart/bronze/lms_attributes/"
bronze_lms_directory_financials = "datamart/bronze/lms_financials/"

for dir_path in [bronze_lms_directory_loan, bronze_lms_directory_clickstream, bronze_lms_directory_attributes, bronze_lms_directory_financials]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        
for date_str in dates_str_lst:
    utils.data_processing_bronze_feature.process_bronze_loan(date_str, bronze_lms_directory_loan, spark)
    utils.data_processing_bronze_feature.process_bronze_clickstream(date_str, bronze_lms_directory_clickstream, spark)
    utils.data_processing_bronze_feature.process_bronze_attributes(date_str, bronze_lms_directory_attributes, spark)
    utils.data_processing_bronze_feature.process_bronze_financials(date_str, bronze_lms_directory_financials, spark)



2023-01-01row count: 530
saved to: datamart/bronze/lms_loan/bronze_loan_daily_2023_01_01.csv


2023-01-01row count: 8974
saved to: datamart/bronze/lms_clickstream/bronze_clickstream_2023_01_01.csv
2023-01-01row count: 530
saved to: datamart/bronze/lms_attributes/bronze_attributes_2023_01_01.csv
2023-01-01row count: 530
saved to: datamart/bronze/lms_financials/bronze_financials_2023_01_01.csv
2023-02-01row count: 1031
saved to: datamart/bronze/lms_loan/bronze_loan_daily_2023_02_01.csv
2023-02-01row count: 8974
saved to: datamart/bronze/lms_clickstream/bronze_clickstream_2023_02_01.csv
2023-02-01row count: 501
saved to: datamart/bronze/lms_attributes/bronze_attributes_2023_02_01.csv
2023-02-01row count: 501
saved to: datamart/bronze/lms_financials/bronze_financials_2023_02_01.csv
2023-03-01row count: 1537
saved to: datamart/bronze/lms_loan/bronze_loan_daily_2023_03_01.csv
2023-03-01row count: 8974
saved to: datamart/bronze/lms_clickstream/bronze_clickstream_2023_03_01.csv
2023-03-01row count: 506
saved to: datamart/bronze/lms_attributes/bronze_attributes_2023_03_01.csv
2023-03-01r

In [4]:

bronze_lms_directory_loan = "datamart/bronze/lms_loan/"
bronze_lms_directory_clickstream = "datamart/bronze/lms_clickstream/"
bronze_lms_directory_attributes = "datamart/bronze/lms_attributes/"
bronze_lms_directory_financials = "datamart/bronze/lms_financials/"

silver_loan_daily_directory = "datamart/silver/loan_daily/"
silver_loan_directory_clickstream = "datamart/silver/loan_clickstream/"
silver_loan_directory_attributes = "datamart/silver/loan_attributes/"
silver_loan_directory_financials = "datamart/silver/loan_financials/"

if not all(os.path.exists(p) for p in [
    silver_loan_daily_directory,
    silver_loan_directory_clickstream,
    silver_loan_directory_attributes,
    silver_loan_directory_financials
]):
    for p in [
        silver_loan_daily_directory,
        silver_loan_directory_clickstream,
        silver_loan_directory_attributes,
        silver_loan_directory_financials
    ]:
        os.makedirs(p, exist_ok=True)

# run silver backfill
for date_str in dates_str_lst:
    utils.data_processing_silver_feature.process_silver_table(date_str, bronze_lms_directory_loan, silver_loan_daily_directory, spark)
    utils.data_processing_silver_feature.process_silver_clickstream(date_str, bronze_lms_directory_clickstream, silver_loan_directory_clickstream, spark)
    utils.data_processing_silver_feature.process_silver_attributes(date_str, bronze_lms_directory_attributes, silver_loan_directory_attributes, spark)
    utils.data_processing_silver_feature.process_silver_financials(date_str, bronze_lms_directory_financials, silver_loan_directory_financials, spark)
    

    


loaded from: datamart/bronze/lms_loan/bronze_loan_daily_2023_01_01.csv row count: 530


saved to: datamart/silver/loan_daily/silver_loan_daily_2023_01_01.parquet
loaded from: datamart/bronze/lms_clickstream/bronze_clickstream_2023_01_01.csv row count: 8974
saved to: datamart/silver/loan_clickstream/silver_clickstream_2023_01_01.parquet
loaded from: datamart/bronze/lms_attributes/bronze_attributes_2023_01_01.csv row count: 530
saved to: datamart/silver/loan_attributes/silver_attributes_2023_01_01.parquet
loaded from: datamart/bronze/lms_financials/bronze_financials_2023_01_01.csv row count: 530
saved to: datamart/silver/loan_financials/silver_financials_2023_01_01.parquet
loaded from: datamart/bronze/lms_loan/bronze_loan_daily_2023_02_01.csv row count: 1031
saved to: datamart/silver/loan_daily/silver_loan_daily_2023_02_01.parquet
loaded from: datamart/bronze/lms_clickstream/bronze_clickstream_2023_02_01.csv row count: 8974
saved to: datamart/silver/loan_clickstream/silver_clickstream_2023_02_01.parquet
loaded from: datamart/bronze/lms_attributes/bronze_attributes_2023_02_0

In [5]:
gold_feature_store_directory = "datamart/gold/feature_store/"
silver_loan_daily_directory = "datamart/silver/loan_daily/"
silver_loan_directory_clickstream = "datamart/silver/loan_clickstream/"
silver_loan_directory_attributes = "datamart/silver/loan_attributes/"
silver_loan_directory_financials = "datamart/silver/loan_financials/"

if not os.path.exists(gold_feature_store_directory):
    os.makedirs(gold_feature_store_directory)

# run gold backfill
for date_str in dates_str_lst:
    utils.data_processing_gold_feature.process_features_gold_table(date_str, silver_loan_daily_directory,  silver_loan_directory_clickstream,silver_loan_directory_attributes, silver_loan_directory_financials,gold_feature_store_directory,spark)

folder_path = gold_feature_store_directory
files_list = glob.glob(os.path.join(folder_path, '*')) 
df = spark.read.option("header", "true").parquet(*files_list)

print("row_count:", df.count())
df.show()

gold table saved to datamart/gold/feature_store/gold_table_2023_01_01.parquet
gold table saved to datamart/gold/feature_store/gold_table_2023_02_01.parquet
gold table saved to datamart/gold/feature_store/gold_table_2023_03_01.parquet
gold table saved to datamart/gold/feature_store/gold_table_2023_04_01.parquet
gold table saved to datamart/gold/feature_store/gold_table_2023_05_01.parquet
gold table saved to datamart/gold/feature_store/gold_table_2023_06_01.parquet
gold table saved to datamart/gold/feature_store/gold_table_2023_07_01.parquet
gold table saved to datamart/gold/feature_store/gold_table_2023_08_01.parquet
gold table saved to datamart/gold/feature_store/gold_table_2023_09_01.parquet
gold table saved to datamart/gold/feature_store/gold_table_2023_10_01.parquet
gold table saved to datamart/gold/feature_store/gold_table_2023_11_01.parquet
gold table saved to datamart/gold/feature_store/gold_table_2023_12_01.parquet
gold table saved to datamart/gold/feature_store/gold_table_2024_

In [6]:
selected_columns = [
    "Customer_ID", "loan_amt","Annual_Income", "Credit_Utilization_Ratio", 
    "Payment_Behaviour", "Num_of_Loan", "Num_of_Delayed_Payment", "fe_3", "fe_7","Changed_Credit_Limit","Outstanding_Debt","Monthly_Balance"
]

final_feature_df = df.select(*selected_columns)
final_feature_df.write.mode("overwrite").parquet("datamart/gold/final_features/")
final_feature_df.show()

+-----------+--------+-------------+------------------------+--------------------+-----------+----------------------+----+----+--------------------+----------------+---------------+
|Customer_ID|loan_amt|Annual_Income|Credit_Utilization_Ratio|   Payment_Behaviour|Num_of_Loan|Num_of_Delayed_Payment|fe_3|fe_7|Changed_Credit_Limit|Outstanding_Debt|Monthly_Balance|
+-----------+--------+-------------+------------------------+--------------------+-----------+----------------------+----+----+--------------------+----------------+---------------+
| CUS_0x1037| 10000.0|    15989.085|                 40.6977|LowspentSmallvalu...|          4|                    15|  80| 111|                 0.5|          665.82|      284.38013|
| CUS_0x1069| 10000.0|     58637.34|               25.233143|HighspentSmallval...|       NULL|                    17| 123|  25|               12.56|           208.8|      434.84888|
| CUS_0x114a| 10000.0|     15305.46|               27.525112|LowspentSmallvalu...|        